# Overview of Prompt Engineering Best Practices

## Part 1: Prompt Engineering Best Practices

In this section, we provide an overview of the top tips and best practices for prompting LLMs.

---

We first load the necessary libraries:

In [ ]:
# ! pip install openai==0.28 --quiet

In [ ]:
# !pip install ollama

In [5]:
# import openai
import os
import IPython

# https://github.com/ollama/ollama-python/tree/main
# import ollama
from ollama import chat

# API configuration
# openai.api_key = "OPENAI_API_KEY"

In [ ]:
# completion function using OpenAI
# def get_completion(messages, model="gpt-3.5-turbo", temperature=0, max_tokens=300):
#     response = openai.ChatCompletion.create(
#         model=model,
#         messages=messages,
#         temperature=temperature,
#         max_tokens=max_tokens,
#     )
#     return response.choices[0].message["content"]

In [8]:
# completion function using ollama

def get_completion(messages, model="mistral", temperature=0, max_tokens=300):
    # response = chat(model=model, messages=messages, temperature=temperature, max_tokens=max_tokens)
    response = chat(model=model, messages=messages)

    return response['message']['content']

""" 
messages example:
[
    {
        'role': 'user',
        'content': 'Why is the sky blue?',
    },
]
"""

" \nmessages example:\n[\n    {\n        'role': 'user',\n        'content': 'Why is the sky blue?',\n    },\n]\n"

### Be Specific and Clear

Write instructions as clear and specific as possible to get the desired LLM behaviors:

In [9]:
global_trending_movies = ["The Suicide Squad", "No Time to Die", "Dune",  "Spider-Man: No Way Home", "The French Dispatch", "Black Widow", "Eternals", "The Matrix Resurrections", "West Side Story", "The Many Saints of Newark"]

system_message = """
Your task is to recommend movies to a customer.

You are responsible to recommend a movie from the top global trending movies from {global_trending_movies}.

You should refrain from asking users for their preferences and avoid asking for personal information.

If you don't have a movie to recommend or don't know the user interests, you should respond "Sorry, couldn't find a movie to recommend today.".
"""

user_request = """
Please recommend a movie based on my interests.
"""

message = [
    {
        "role": "system",
        "content": system_message.format(global_trending_movies=global_trending_movies)
    },
    {
        "role": "user",
        "content": user_request
    }
]

response = get_completion(message)
print(response)

 Based on the list of top global trending movies, I would recommend "The Suicide Squad". It is a superhero action film that features an ensemble cast of DC Comics characters. If you enjoy action-packed movies with a mix of humor and drama, then "The Suicide Squad" might be a great choice for you.


The more specific the desired the behavior you want from the model, the more specific the instructions and logic should be. Below is an example where the customer provides information about interests:

In [10]:
global_trending_movies = ["The Suicide Squad", "No Time to Die", "Dune",  "Spider-Man: No Way Home", "The French Dispatch", "Black Widow", "Eternals", "The Matrix Resurrections", "West Side Story", "The Many Saints of Newark"]

system_message = """
Your task is to recommends movies to a customer.

You are responsible to recommend a movie from the top global trending movies from {global_trending_movies}.

You should refrain from asking users for their preferences and avoid asking for personal information.

If you don't have a movie to recommend or don't know the user interests, you should respond "Sorry, couldn't find a movie to recommend today.".
"""

user_request = """
I love super-hero movies. Please recommend a movie based on my interests.
"""

message = [
    {
        "role": "system",
        "content": system_message.format(global_trending_movies=global_trending_movies)
    },
    {
        "role": "user",
        "content": user_request
    }
]

response = get_completion(message)
print(response)

 Based on your interest in superhero movies, I would highly recommend "The Suicide Squad" or "Eternals" for you. Both movies feature unique and compelling stories with larger-than-life characters that are sure to keep you engaged. Enjoy your movie experience!


### Add Delimiters

Adding delimiters help to better structure instructions and the overall prompt components. This is beneficial to get more reliable responses.

In [11]:
prompt = """
Convert the following code block in the #### <code> #### section to Python:

####
strings2.push("one")
strings2.push("two")
strings2.push("THREE")
strings2.push("4")
####
"""

message = [
    {
        "role": "user",
        "content": prompt
    }
]

IPython.display.Markdown("```python" + get_completion(message) + "\n```")

```python In Python, you can use a list instead of an array-like data structure provided by JavaScript. Here's the equivalent code snippet in Python:

```python
strings2 = []
strings2.append("one")
strings2.append("two")
strings2.append("THREE")
strings2.append("4")
```
```

### Specify Output Format

If the format of prompt responses are important, then this should be explicitly stated in the prompt to get desired results. In the example, we would like to export the results as a JSON object.

In [12]:
prompt = """
Your task is: given a product description, return the requested information in the section delimited by ### ###. Format the output as a JSON object.

Product Description: Introducing the Nike Air Max 270 React: a comfortable and stylish sneaker that combines two of Nike's best technologies. With a sleek black design and a unique bubble sole, these shoes are perfect for everyday wear.

###
product_name: the name of the product
product_bran: the name of the brand (if any)
###
"""

message = [
    {
        "role": "user",
        "content": prompt
    }
]

print(get_completion(message))

 {
  "product\_name": "Nike Air Max 270 React",
  "product\_brand": "Nike"
}


### Think Step by Step

To elicit reasoning in LLMs, you can prompt the model to think step-by-step. Prompting the model in this way allows it to provide the details steps before providing a final response that solves the problem.

In [13]:
prompt = """The odd numbers in this group add up to an even number: 15, 32, 5, 13, 82, 7, 1.

Solve by breaking the problem into steps. First, identify the odd numbers, add them, and indicate whether the result is odd or even."""

messages = [
    {
        "role": "system",
        "content": prompt
    }
]

response= get_completion(messages)

print(response)

 Step 1: Identify the odd numbers in the given group: 15, 32, 5, 13, 82, 7, 1.
The odd numbers are: 1, 5, 13.

Step 2: Add up the identified odd numbers:
1 + 5 + 13 = 19

Step 3: Determine if the result is odd or even:
Since 19 is a positive integer that can be written as an even number (2 x 9) plus an odd number (1), it is itself an odd number.

So, the sum of the odd numbers in this group is an odd number (19).


### Role Playing

The example below shows how to apply role playing using a chat model like GPT-3.5 Turbo/Mistral. Notice the use of system message, user message, and assistant message in the example. You can combine different messages to mimic or jump start the behavior you want or expect from the model.

In [14]:
system_message = """
The following is a conversation with an AI research assistant. The assistant tone is technical and scientific.
"""

user_message_1 = """
Hello, who are you?
"""

ai_message_1 = """
Greeting! I am an AI research assistant. How can I help you today?
"""

prompt = """
Human: Can you tell me about the creation of blackholes?
AI:
"""

messages = [
    {
        "role": "system",
        "content": system_message
    },
    {
        "role": "user",
        "content": user_message_1
    },
    {
        "role": "assistant",
        "content": ai_message_1

    },
    {
        "role": "user",
        "content": prompt
    }
]

response = get_completion(messages)
print(response)

 Absolutely! A black hole is a region in space where gravity is so strong that nothing, not even light, can escape its pull when it passes the event horizon. Black holes are formed from massive stars that have exhausted their nuclear fuel and undergone a supernova explosion, causing their cores to collapse in on themselves.

The process begins when a star with at least three times the mass of our sun reaches the end of its life cycle. The core collapses under the force of gravity, resulting in an extremely dense object - an object whose gravitational pull is so strong that not even light can escape it, hence the name "black hole."

The exact mechanism behind this collapse and the formation of the event horizon, which marks the boundary between the black hole and the rest of the universe, is still an area of ongoing research in physics. However, we have a good understanding of the basic principles involved.

Black holes can be characterized by their mass, spin, and charge. They can also